In [1]:
# Load generated recipes files
import pandas as pd
from tensorflow.python.ops.metrics_impl import false_positives_rag

# Load the data
manual = pd.read_json('../data/generated_recipes_manual_review.json', dtype={'correct': 'boolean'})
rag = pd.read_json('../data/generated_recipes_rag_review.json', dtype={'correct': 'boolean'})
moderator = pd.read_json('../data/generated_recipes_moderator_review.json', dtype={'correct': 'boolean'})
combined = pd.read_json('../data/generated_recipes_combined_review.json', dtype={'correct': 'boolean'})
manual['correct'] = manual['correct'].fillna(False)

rag.rename(columns={'correct': 'correct_rag'}, inplace=True)
moderator.rename(columns={'correct': 'correct_moderator'}, inplace=True)
combined.rename(columns={'correct': 'correct_combined'}, inplace=True)
totalset = pd.merge(manual, rag[['medische_vraag','correct_rag']], on='medische_vraag', how='left')
totalset = pd.merge(totalset, moderator[['medische_vraag','correct_moderator']], on='medische_vraag', how='left')
totalset = pd.merge(totalset, combined[['medische_vraag','correct_combined']], on='medische_vraag', how='left')


,medicijn,categorie,recept,medische_vraag,correct,correct_rag,correct_moderator,correct_combined
0,Amoxicilline,Antibioticum,Rp. Amoxicilline 500mg capsule\ndtd. No. 21\nS...,Ik heb een keelontsteking. Heb ik antibiotica ...,True,True,True,True
1,nitrofurantoïne,antibioticum,Rp. Nitrofurantoïne 100mg capsules\ndtd. No. 2...,Ik heb blaasontstekingklachten. Wat kan ik daa...,True,True,True,True
2,Amoxicilline,Antibioticum,Rp. Amoxicilline 500mg tabletten\ndtd. No. 20\...,Ik heb een longontsteking. Welke antibiotica z...,True,True,True,True
3,Amoxicilline/Clavulaanzuur,Antibioticum,Rp. Amoxicilline/Clavulaanzuur 500/125mg table...,Ik heb wondinfectieverschijnselen. Welk antibi...,False,True,True,True
4,Amoxicilline,Antibioticum,Rp. Amoxicilline 500mg capsule \ndtd. No. 21\n...,Ik heb een tandvleesontsteking. Kan ik hier ie...,True,False,True,False
...,...,...,...,...,...,...,...,...
194,Glycopyrroniumbromide,Anticholinergica,Rp. Glycopyrroniumbromide 1mg tablet\ndtd. No....,Ik heb slikproblemen. Zijn er medicijnen om de...,False,True,True,True
195,Baclofen,Spierrelaxantia,Rp. Baclofen 10mg tablet\ndtd. No. 50\nS. Drie...,Ik heb spastische spieren door MS. Wat kan ik ...,False,True,True,True
196,Dimethylfumaraat,Multiple Sclerose Medicatie,Rp. Dimethylfumaraat 120mg capsule\ndtd. No. 1...,Ik heb MS. Wat zijn mijn onderhoudsmedicijnen?,False,True,True,True
197,Levothyroxine,Schildklierhormoon,Rp. Levothyroxine 100mcg tablet\ndtd. No. 30\n...,Ik heb een vergrote schildklier. Moet ik iets ...,False,True,True,True


In [6]:
# Count the number of correct answers
correct_counts = totalset[['correct', 'correct_rag', 'correct_moderator', 'correct_combined']].sum()
# Print the counts
print("Number of correct answers:")
print(correct_counts)

false_positives_rag = totalset[(totalset['correct'] == False) & (totalset['correct_rag'] == True)]['medische_vraag'].count()
false_positives_moderator = totalset[(totalset['correct'] == False) & (totalset['correct_moderator'] == True)]['medische_vraag'].count()
false_positives_combined = totalset[(totalset['correct'] == False) & (totalset['correct_combined'] == True)]['medische_vraag'].count()

print(f"Number of false positives: {false_positives_rag}")
print(f"Number of false positives moderator: {false_positives_moderator}")
print(f"Number of false positives combined: {false_positives_combined}")

Number of correct answers:
correct               19
correct_rag          185
correct_moderator    196
correct_combined     185
dtype: Int64
Number of false positives: 167
Number of false positives moderator: 177
Number of false positives combined: 167
